In [ ]:
import pandas as pd
from PIL import Image
import numpy as np
from tqdm.notebook import tqdm
import cv2 
from tacobox import Taco
import random

In [ ]:
# train_df_upti = pd.read_csv("data_upti/train.csv")
# train_df_ticker = pd.read_csv("data_ticker/train.csv")
# test_df_upti = pd.read_csv("data_upti/test.csv")
# test_df_ticker = pd.read_csv("data_ticker/test.csv")

# train_df = pd.read_csv("data_ihtr/trainingset/urdu/data.csv")
# eval_df = pd.read_csv("data_ihtr/validationset/urdu/data.csv")


# train_df_icdar = pd.read_csv("data_upti/train_icdar.csv")
train_df = pd.read_csv("data_uhwr/train.csv")
# train_df = pd.read_csv
# eval_df = pd.read_csv("data_upti/test_icdar.csv")
eval_df = pd.read_csv("data_uhwr/val.csv")
test_df = pd.read_csv("data_uhwr/test.csv")
# books = pd.read_csv("./data_urti/books.csv")
# magazines = pd.read_csv("./data_urti/magazines.csv")
# novels = pd.read_csv("./data_urti/novels.csv")
# poetry = pd.read_csv("./data_urti/poetry.csv")

# df = ()

In [ ]:
# train_df_uhwr

In [ ]:
# df_1 = pd.read_csv("MMU-OCR-21/TextLines/UrduTextLineNastaleeqOutput.csv")[['ImagePath', 'UrduLines']].rename(columns={"ImagePath": "file_name", "UrduLines":"text"})
# df_2 = pd.read_csv("MMU-OCR-21/TextLines/UrduTextLinesNaskhOutput.csv")[['ImagePath', 'UrduLines']].rename(columns={"ImagePath": "file_name", "UrduLines":"text"})
# df_3 = pd.read_csv("MMU-OCR-21/TextLines/UrduTextLinesTehreerOutput.csv")[['ImagePath', 'UrduLines']].rename(columns={"ImagePath": "file_name", "UrduLines":"text"})

In [ ]:
# train_df_icdar = train_df_icdar.sample(frac=1).reset_index(drop=True)

In [ ]:
# train_df = pd.concat([train_df_icdar[:len(train_df_uhwr)], train_df_uhwr]).reset_index(drop=True).sample(frac=1).reset_index(drop=True)

In [ ]:
# train_df = pd.read_csv("data_uhwr/train.csv")
# eval_df = pd.read_csv("data_uhwr/val.csv")
# test_df = pd.reaad_csv('data_uhwr/test.csv')

In [ ]:
# train_df = pd.concat([train_df_upti, train_df_ticker]).reset_index().drop("index", axis=1)
# test_df = pd.concat([test_df_upti, test_df_ticker]).reset_index().drop("index", axis=1)

In [ ]:
train_df

In [ ]:
eval_df

In [ ]:
# books

In [ ]:
# train_df['text'].to_csv("corpus.txt", index=False, header=False)

In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor

class HWRDataset(Dataset):
    def __init__(self, df, tokenizer, input_width=1600, 
                 input_height=64,
                 aug = False,
                 taco_aug_frac=0.9):
        self.df = df
        self.input_width = input_width
        self.input_height = input_height
        self.tokenizer = tokenizer
        self.mytaco = Taco(
            cp_vertical=0.2,
            cp_horizontal=0.25,
            max_tw_vertical=100,
            min_tw_vertical=10,
            max_tw_horizontal=50,
            min_tw_horizontal=10
        )
        self.aug=aug
        self.taco_aug_frac=taco_aug_frac

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # get file name + text 
        file_name = self.df['file_name'][idx]
        text = self.df['text'][idx]
        
        image = cv2.imread(file_name, cv2.IMREAD_GRAYSCALE)
        
        pixel_values = self.preprocess(image, self.aug)

        # image = cv2.resize(image, (512, 64), cv2.INTER_AREA)

        # pixel_values = Image.fromarray(image.T)
        try:
            labels = self.tokenizer(text).input_ids
        except:
            labels = None
            encoding = (None, None)
        else:
            labels = [self.tokenizer.bos_token_id] + [label if label != self.tokenizer.pad_token_id else -100 for label in labels]+[self.tokenizer.eos_token_id]
            encoding = (torch.tensor(pixel_values[None,:,:]).float(), torch.tensor(labels))
        
        return encoding

    def preprocess(self, img, augment=True):
        if augment:
            img = self.apply_taco_augmentations(img)
            
        # scaling image [0, 1]
        img = img/255
        img = img.swapaxes(-2,-1)[...,::-1]
        # img = img.swapaxes(-2,-1)
        target = np.ones((self.input_width, self.input_height))
        # target = np.zeros((self.input_width, self.input_height))
        new_x = self.input_width/img.shape[0]
        new_y = self.input_height/img.shape[1]
        min_xy = min(new_x, new_y)
        new_x = int(img.shape[0]*min_xy)
        new_y = int(img.shape[1]*min_xy)
        img2 = cv2.resize(img, (new_y,new_x))
        target[:new_x,:new_y] = img2
        return 1 - (target)
        # return (target)
        #89 1661

    def apply_taco_augmentations(self, input_img):
        random_value = random.random()
        if random_value <= self.taco_aug_frac:
            augmented_img = self.mytaco.apply_vertical_taco(
                input_img, 
                corruption_type='random'
            )
        else:
            augmented_img = input_img
        return augmented_img

In [ ]:
# from tokenizers import ByteLevelBPETokenizer
# # initialize
# tokenizer = ByteLevelBPETokenizer()
# # and train
# tokenizer.train(files="corpus.txt", vocab_size=256, min_frequency=2,
#                 special_tokens=['<s>', '<pad>', '</s>', '<unk>', '<mask>'])

# tokenizer.save_model("vocabs/ved/")

In [ ]:
from transformers import RobertaTokenizerFast, ViTFeatureExtractor, BertTokenizer
from transformers import GPT2Tokenizer

# tokenizer = RobertaTokenizerFast.from_pretrained("vocabs/ved/")
tokenizer = GPT2Tokenizer.from_pretrained("vocabs/ved/")


# tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-uncased")

# feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")

tokenizer.bos_token = '<s>'
tokenizer.eos_token = '</s>'
tokenizer.pad_token = '<pad>'
tokenizer.unk_token = '<unk>'


train_dataset = HWRDataset(df=train_df,
                           tokenizer=tokenizer,
                           aug=True
                           )
eval_dataset = HWRDataset(df=eval_df,
                         tokenizer=tokenizer,)

# test_dataset = HWRDataset(df=test_df,
#                           tokenizer=tokenizer)

# books = HWRDataset(df=books,
#                    tokenizer=tokenizer,)
                         
# magazines = HWRDataset(df=magazines,
#                        tokenizer=tokenizer,)

# novels = HWRDataset(df=novels,
#                     tokenizer=tokenizer,)

# poetry = HWRDataset(df=poetry,
#                     tokenizer=tokenizer,)                        

In [ ]:
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(eval_dataset))
# print("Number of books examples:", len(books))
# print("Number of magazines examples:", len(magazines))
# print("Number of novels examples:", len(novels))
# print("Number of poetry examples:", len(poetry))

In [ ]:
N = 3

# encoding = train_dataset[N]
# for k,v in encoding.items():
#   print(k, v.shape)

encoding = train_dataset[N]
print(encoding[0].shape, encoding[1].shape)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

figure(figsize=(10, 10), dpi=80)

# plt.imshow(encoding['pixel_values'].permute(1,2,0))
plt.imshow(encoding[0].permute(1,2,0), cmap="gray")
# plt.imshow(encoding[0], cmap="gray")

In [ ]:
image = Image.open(train_df['file_name'][N]).convert("RGB")
image

In [ ]:
# labels = encoding['labels']
labels = encoding[1]
labels[labels == -100] = tokenizer.pad_token_id
label_str = tokenizer.decode(labels, skip_special_tokens=True)
print(label_str)

In [ ]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    src_batch, tgt_batch = [], []
    batch_dict = {}
    for src_sample, tgt_sample in batch:
        if src_sample == None:
            continue
        if tgt_sample == None:
            continue
        src_batch.append(src_sample) #ADDEDLINE
        tgt_batch.append(tgt_sample)


    src_batch = torch.stack(src_batch)

    tgt_batch = pad_sequence(tgt_batch, batch_first=True, padding_value=-100)

    batch_dict['pixel_values'] = src_batch
    batch_dict['labels'] = tgt_batch
    return batch_dict

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)
eval_dataloader = DataLoader(eval_dataset, batch_size=16, collate_fn=collate_fn)
# test_dataloader = DataLoader(test_dataset, batch_size=16, collate_fn=collate_fn)
# books_dataloader = DataLoader(books, batch_size=16, collate_fn=collate_fn)
# magazines_dataloader = DataLoader(magazines, batch_size=16, collate_fn=collate_fn)
# novels_dataloader = DataLoader(novels, batch_size=16, collate_fn=collate_fn)
# poetry_dataloader = DataLoader(poetry, batch_size=16, collate_fn=collate_fn)

In [ ]:
for x in tqdm(train_dataloader):
    print(x['labels'])
    break

In [ ]:
# from transformers import VisionEncoderDecoderModel, RobertaConfig, RobertaForCausalLM, ViTModel
# import torch
from transformers import EncoderDecoderModel
from models import *



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



conv, transformer = model_conv_transformer(inchannel=256, vocab_size=tokenizer.vocab_size)

# transformer = EncoderDecoderModel.from_pretrained("conv_transformer_weights/icdar/")


# conv.load_state_dict(torch.load("conv_transformer_weights/icdar/conv.pt"))

conv.to(device)
transformer.to(device)

In [ ]:
transformer.config.decoder_start_token_id = tokenizer.bos_token_id
transformer.config.pad_token_id = tokenizer.pad_token_id
# make sure vocab size is set correctly
transformer.config.vocab_size = transformer.config.decoder.vocab_size

# set beam search parameters
transformer.config.eos_token_id = tokenizer.eos_token_id
transformer.config.max_length = 256
# transformer.config.max_new_tokens = 256 
transformer.config.early_stopping = False
transformer.config.no_repeat_ngram_size = 0
transformer.config.length_penalty = 1
transformer.config.num_beams = 4
transformer.config.temperature = 1

In [ ]:
def train(enc, dec, data_loader, optimizer, ctc_loss):
  enc.train()
  dec.train()
  train_loss = 0.0
  train_loss_cross = 0.0
  train_loss_ctc = 0.0
  correct_train = 0.0
  total_train = 0.0
  train_acc = 0
  for batch in tqdm(data_loader):
    for k,v in batch.items():
      batch[k] = v.to(device)

    pixel_values = batch['pixel_values']
    labels = batch['labels']


    # pixel_values = transform(pixel_values)
    emb = enc(pixel_values)
    

    # pixel_values = conv(pixel_values)
    # loss_ctc = ctc_loss(pred_ctc, labels, input_lengths, output_lengths)

    # logits, loss_cross_entropy = dec(tgt=labels[:, :-1], memory=emb, calc_loss=True, labels = labels[:,1:], tgt_mask=tgt_mask, tgt_key_padding_mask=tgt_key_padding_mask)
    outputs = transformer(inputs_embeds=emb, labels=labels)
    logits = outputs.logits
    loss = outputs.loss
    # loss = outputs.loss

    # loss = (loss_cross_entropy+loss_ctc)/2
    # loss = loss_ctc
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    train_loss += loss.item()
    # train_loss_ctc += loss_ctc.item()
    # train_loss_cross += loss_cross_entropy.item()

    with torch.no_grad():

      preds = torch.argmax(logits, axis=-1)
      mask = torch.ones_like(labels).to(device)
      mask[labels==-100] = 0
      correct_train += ((preds == labels)*mask).sum()
      total_train += mask.sum()
    

  
  train_loss = train_loss/len(train_dataloader)
  train_loss_cross = train_loss_cross/len(train_dataloader)
  train_loss_ctc = train_loss_ctc/len(train_dataloader)
  train_acc = (correct_train/total_train).to('cpu').item()


  return train_loss, train_loss_cross, train_loss_ctc, train_acc


def eval(enc, dec, data_loader, ctc_loss):
  eval_loss = 0.0
  eval_loss_cross = 0.0
  eval_loss_ctc = 0.0
  correct_eval = 0.0
  total_eval = 0.0
  eval_acc=0
  enc.eval()
  dec.eval()
  with torch.no_grad():
    for batch in tqdm(data_loader):
      for k,v in batch.items():
        batch[k] = v.to(device)

      pixel_values = batch['pixel_values']
      labels = batch['labels']

      # pixel_values = transform(pixel_values)
      emb = enc(pixel_values)
      # print(emb.shape)

      # pixel_values = conv(pixel_values)
      # loss_ctc = ctc_loss(pred_ctc, labels, input_lengths, output_lengths)
      # print(tgt_mask.shape)
      # print(tgt_key_padding_mask.shape)
      # print(labels[:, 1:].shape)
      # logits, loss_cross_entropy = dec(tgt=labels[:, :-1], memory=emb, calc_loss=True, labels = labels[:,1:], tgt_mask=tgt_mask, tgt_key_padding_mask=tgt_key_padding_mask)
      # print(a)
      outputs = transformer(inputs_embeds=emb, labels=labels)
      logits = outputs.logits
      loss = outputs.loss
      # loss = outputs.loss

      # loss = (loss_cross_entropy+loss_ctc)/2
      # loss = loss_ctc
      eval_loss += loss.item()
      # eval_loss_cross += loss_cross_entropy.item()
      # eval_loss_ctc += loss_ctc.item()

      preds = torch.argmax(logits, axis=-1)
      mask = torch.ones_like(labels).to(device)
      mask[labels==-100] = 0
      correct_eval += ((preds == labels)*mask).sum()
      total_eval += mask.sum()


    eval_loss = eval_loss/len(eval_dataloader)
    eval_loss_cross = eval_loss_cross/len(eval_dataloader)
    eval_loss_ctc = eval_loss_ctc/len(eval_dataloader)
    eval_acc = (correct_eval/total_eval).to('cpu').item()

  return eval_loss, eval_loss_cross, eval_loss_ctc, eval_acc

In [ ]:
loss_fn = torch.nn.CTCLoss(blank=tokenizer.mask_token_id, zero_infinity=True)

from tqdm.notebook import tqdm

params = list(conv.parameters()) + list(transformer.parameters())
# params = list(transformer.parameters())

optimizer = torch.optim.AdamW(params, lr=0.0003)
# optimizer = torch.optim.AdamW(params, lr=0.0003)
# optimizer = torch.optim.Adam(params, lr=0.0003, betas=(0.9, 0.98), eps=1e-9)

num_epochs = 100
num_training_steps = num_epochs * len(train_dataloader)
num_eval_steps = num_epochs * len(eval_dataloader)

print(num_training_steps)

In [ ]:
prev_eval_loss, _, _, prev_eval_acc = eval(conv, transformer,
                                           eval_dataloader, 
                                           loss_fn)
print(prev_eval_loss, prev_eval_acc)

In [ ]:
count = 0
for epoch in range(num_epochs):  # loop over the dataset multiple times

  if count > 1:
    break

  train_loss, train_loss_cross, train_loss_ctc, train_acc = train(conv, transformer,
                                                                  train_dataloader, 
                                                                  optimizer, loss_fn)
  print(f"Train Loss after epoch {epoch}:", train_loss)
  print(f"Train Loss_CROSS after epoch {epoch}:", train_loss_cross)
  print(f"Train Loss_CTC after epoch {epoch}:", train_loss_ctc)
  print(f"Train ACC after epoch {epoch}:", train_acc*100)
  
  eval_loss, eval_loss_cross, eval_loss_ctc, eval_acc = eval(conv, transformer,
                                                              eval_dataloader, 
                                                              loss_fn)
  print(f"VAL P Loss after epoch {epoch}:", eval_loss)
  # print(f"VAL P Loss_CROSS after epoch {epoch}:", eval_loss_cross)
  # print(f"VAL P Loss_CTC after epoch {epoch}:", eval_loss_ctc)
  print(f"VAL P ACC after epoch {epoch}:", eval_acc*100)


  # eval_loss, eval_loss_cross, eval_loss_ctc, eval_acc = eval(conv, transformer,
  #                                                             test_dataloader, 
  #                                                             loss_fn)
  # print(f"VAL H Loss after epoch {epoch}:", eval_loss)
  # # print(f"VAL H Loss_CROSS after epoch {epoch}:", eval_loss_cross)
  # # print(f"VAL H Loss_CTC after epoch {epoch}:", eval_loss_ctc)
  # print(f"VAL H ACC after epoch {epoch}:", eval_acc*100)


  # if prev_eval_acc < eval_acc:
  #   prev_eval_acc = eval_acc
  #   # transformer.save_pretrained("./weights/convnext/")
  #   transformer.save_pretrained("./conv_transformer_weights/icdar")
  #   torch.save(conv.state_dict(), "./conv_transformer_weights/icdar/conv.pt")
  #   print("WEIGHTS SAVED")
  
  # else:
  #   count+=1

In [ ]:
from jiwer import cer
from evaluate import load
cer = load("cer")



def compute_cer(pred_ids, label_ids):



    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)

    label_ids[label_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    print(pred_str[0])
    print(label_str[0])

    # print(label_str)

    combine = [(x, y) for x, y in zip(pred_str, label_str) if x]

    pred_str = [x for x, y in combine]
    label_str = [y for x, y in combine]


    

    # print(pred_str[0])
    # print(label_str[0])

    # if not pred_str:
    #     return 0


    # error = cer(pred_str, label_str)
    cer_score = cer.compute(predictions=pred_str, references=label_str)


    return cer_score

In [ ]:
# device = 'cpu'
# conv.to(device)
# transformer.to(device)
conv.eval()
transformer.eval()
test_cer = 0.0
data_loader = eval_dataloader
with torch.no_grad():
  for batch in tqdm(data_loader):
    # run batch generation
    outputs = conv(batch["pixel_values"].to(device))
    outputs = transformer.generate(inputs_embeds=outputs)
    # compute metrics|
    error = compute_cer(pred_ids=outputs, label_ids=batch["labels"])
    test_cer += error 

print("Validation CER:", test_cer / len(data_loader)*100)